In [ ]:
import pickle
import joblib
import openpyxl
import numpy as np
import pandas as pd
from pathlib import Path

from scipy.signal import welch
from scipy.stats import skew, kurtosis as kurt, entropy as entr

from sklearn.preprocessing import StandardScaler

In [ ]:
sf = 128 # sample rate (Hz)
bins = 10 # entropy calculation
folder_path = Path("..")/"eeg_emotion_data" # folder containing .dat file for each participant

Function to load one participant file

In [ ]:
def load_file(file_path):
    file_path = Path(file_path) # ensure it is a path object

    if not file_path.exists():
        print(f"❌ File not found: {file_path}")
        return None
    
    try:
        with file_path.open("rb") as f:
            participant_data = pickle.load(f, encoding="latin1")

        # extract first 32 channels out of 40 and remove 3-sec pre-trial baseline from each channel
        eeg_data = participant_data["data"][:, :32, 3*sf:] # shape (40, 32, 63*sf - 3*sf) = (trials, channels, sample values)
        labels = participant_data["labels"][:, :2] # shape (40, 2) = (trials, [valence arousal])

        return eeg_data, labels
    
    except (pickle.UnpicklingError, EOFError) as e:
        print(f"❌ Error loading pickle file: {file_path} -> {e}")
        return None
    
    except Exception as e:
        print(f"❌ Unexpected error while loading {file_path}: {e}")
        return None

Function to divide trials of one participant into 10-sec windows

In [ ]:
# Used for creating 7680x321 shaped dataset

# for one participant,
    # rows = trials * no. of windows per trial (40 * 6) [each trial is of one minute]

def windowing(eeg_data, labels):
    new_eeg_data = []
    new_labels = []

    for trial, label_row in zip(eeg_data, labels):
        for i in range(0, 6):
            window = trial[:, (10*i)*sf:(10*(i + 1))*sf] # shape (32, 5*sf)
            new_eeg_data.append(window)
            new_labels.append(label_row)

    return np.array(new_eeg_data), np.array(new_labels) # shape (240, 32, 5*sf), shape (240, 2)

Prepare labels for target variable

In [ ]:
def calculate_focus_level(valence, arousal):
    if valence >= 4.5 and arousal >= 4.5: # highly focused
        return 1
    else: # distracted
        return 0

In [ ]:
def prepare_labels(labels):
    focus_label = np.array([calculate_focus_level(v, a) for v, a in labels])

    return focus_label

Functions to extract bandpower and statistical features

In [ ]:
def compute_bandpower(eeg_channel_arr, band_name):
    bands = {
        "theta": (4, 8),
        "alpha": (8, 12),
        "beta": (12, 30),
        "gamma": (30, 45)
    }

    range = bands[band_name]

    # get array of freqs and corresponding power values
    freqs, psd = welch(eeg_channel_arr, sf, nperseg=256)
    
    # choose freqs within the freq range
    valid_indices = np.logical_and(freqs >= range[0], freqs < range[1])

    # calculate mean of all the power values for one channel
    return np.mean(psd[valid_indices])

In [ ]:
def compute_stat_features(eeg_channel_arr):
    mean = np.mean(eeg_channel_arr)
    std = np.std(eeg_channel_arr)
    skewness = skew(eeg_channel_arr)
    kurtosis = kurt(eeg_channel_arr)

    hist, _ = np.histogram(eeg_channel_arr, bins=bins, density=True)
    entropy = entr(hist)

    return mean, std, skewness, kurtosis, entropy

Function to create feature matrix for one participant

In [ ]:
def create_feature_matrix(eeg_data):
    feature_matrix = []

    for trial in eeg_data:
        trial_features = []

        for channel in trial:
            theta = compute_bandpower(channel, "theta")
            alpha = compute_bandpower(channel, "alpha")
            beta = compute_bandpower(channel, "beta")
            gamma = compute_bandpower(channel, "gamma")

            beta_alpha_ratio = beta/alpha

            mean, std, skewness, kurtosis, entropy = compute_stat_features(channel)

            channel_features = [theta, alpha, beta, gamma, beta_alpha_ratio, mean, std, skewness, kurtosis, entropy]

            trial_features.extend(channel_features)

        feature_matrix.append(trial_features)

    return np.array(feature_matrix)

Function to create a single featre matrix for all participants

In [ ]:
def final_matrix(folder_path, windowing_used):
    feature_matrix, focus_label = [], []

    for participant_file in folder_path.iterdir():
        eeg_data, labels = load_file(participant_file)

        if eeg_data is None or labels is None:
            print(f"⚠️ Skipping {participant_file}: Failed to load data")
            continue

        if windowing_used == True:
            eeg_data, labels = windowing(eeg_data, labels)

        focus_label.extend(prepare_labels(labels).tolist())

        feature_matrix.append(create_feature_matrix(eeg_data)) # creates list of matrices

    feature_matrix = np.concatenate(feature_matrix, axis=0) # list of matrices to ndarray
    focus_label = np.array(focus_label).reshape(-1, 1) # convert labels into a column vector

    return np.hstack((feature_matrix, focus_label))

Function to save the final matrix as a DataFrame

In [ ]:
def save_mat_as_df(matrix, path):
    features = ["Theta", "Alpha", "Beta", "Gamma", "BetaAlpha", "Mean", "Std", "Skew", "Kurt", "Entropy"]

    column_names = [f"{feature}_{i+1}" for i in range(32) for feature in features]
    column_names.append("Focus_Level")

    df = pd.DataFrame(matrix, columns=column_names)
    df.to_excel(path, index=False)

    return df

#### Create a dataset with each trial as a row

In [ ]:
feature_matrix = final_matrix(folder_path, False)

Standardization

In [ ]:
scalar = StandardScaler()
feature_matrix[:, :-1] = scalar.fit_transform(feature_matrix[:, :-1]) # exclude the target variable
joblib.dump(scalar, '../models/scalar_1d.pkl')

Save the matrix as a DataFrame

In [ ]:
path = "../1280x321.xlsx"
df = save_mat_as_df(feature_matrix, path)
df

##### Create a dataset with each 10-sec window as a row

In [ ]:
feature_matrix = final_matrix(folder_path, True)

Standardization

In [ ]:
scalar = StandardScaler()
feature_matrix[:, :-1] = scalar.fit_transform(feature_matrix[:, :-1]) # exclude the target variable
joblib.dump(scalar, '../models/scalar_2d.pkl')

Save the matrix as a DataFrame

In [ ]:
path = "../7680x321.xlsx"
df = save_mat_as_df(feature_matrix, path)
df